<a href="https://colab.research.google.com/github/Graur/mipt/blob/master/%D0%93%D1%80%D0%B0%D1%83%D1%80_%D0%90%D0%BD%D0%B4%D1%80%D0%B5%D0%B9_%D0%9A%D0%BE%D0%BD%D1%81%D1%82%D0%B0%D0%BD%D1%82%D0%B8%D0%BD%D0%BE%D0%B2%D0%B8%D1%87_%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B8%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F_%D0%A1%D0%BE%D1%80%D0%B5%D0%B2%D0%BD%D0%BE%D0%B2%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B8__DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# План домашних заданий


- Описание задачи и знакомство с данными
- Домашнее задание 1. Генерация и фильтрация признаков
- Домашнее задание 2. Прогноз времени и вида поломки машин, настройка ML-модели
- Дополнительное задание. Визуализация прогнозов, ошибок модели и важности признаков
- Домашнее задание 3. Оптимизация. Тюнинг гиперпараметров с помощью `Optuna`
- Домашнее задание 4. Блендинг
- Домашнее задание 5. Парсинг внешних данных и оптимизация памяти
- Отправка финального решения на лидерборд на Kaggle

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# <center id="part0"> Описание задачи

**Предыстория:**  мы работаем с каршеринговой компанией, которая управляет крупным автопарком машин. Наша цель — предотвратить длительные периоды простоя машин из-за поломок через своевременное обслуживание и ремонт.


**Идея для решения проблемы:** создать приоритизированный список обхода машин. Этот список поможет технической бригаде сосредоточиться на автомобилях, которые наиболее вероятно выйдут из строя в ближайшее время.


**Как компания решает задачу:**

* Собирает данные о поездках и состоянии машин до поломок.

* Нанимает Data Scientist, чтобы он смог использовать эти данные для анализа и прогнозирования характера поломок.

**Важный момент**: задачи этого специалиста (Data Scientist) предстоит выполнять вам.

## <center> Знакомство с данными

### Информация про машины с таргетом (основной датасет)

In [ ]:
car_train = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/car_train.csv')
car_train.sample(5)

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
1661,T19756755G,Tesla Model 3,premium,electro,3.12,2017,135804,2018,120.52,another_bug
2081,Q-3432704o,VW Tiguan,economy,petrol,4.82,2016,95302,2019,57.08,engine_overheat
291,i-2453146k,Skoda Rapid,economy,petrol,4.02,2015,79308,2017,51.12,engine_overheat
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,35.20,electro_bug
1173,j18260546M,Fiat 500,business,petrol,3.38,2013,44911,2021,30.51,engine_overheat


- `car_id` — идентификатор машины
- `model` / `car_type` / `fuel_type` — марка, класс и тип топлива машины
- `car_rating` / `riders` — общий рейтинг и общее число поездок к концу 2021 года
- `year_to_start` / `year_to_work` — год выпуска машины и год начала работы в автопарке
- `target_reg` — количество дней до поломки
- `target_class` — класс поломки (всего 9 видов)

### Информация про поездки

In [ ]:
rides_info = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/rides_info.csv')
rides_info.sample(5)

,user_id,car_id,ride_id,ride_date,rating,ride_duration,ride_cost,speed_avg,speed_max,stop_times,distance,refueling,user_ride_quality,deviation_normal
519004,m73049370a,k15728070F,Z1t,2020-03-13,3.60,27,237,39,75.000000,1,1028.451629,0,19.438265,-20.628
581922,f22378022p,p-2236312n,D1z,2020-02-05,2.73,28,273,48,66.000000,1,1249.331223,0,2.543207,3.305
126390,I64402647h,I58543255u,s1V,2020-02-05,4.14,39,385,42,71.094413,1,1469.223338,0,-18.433424,2.553
437299,P73311585i,f-1056352e,S1k,2020-01-20,7.15,26,410,33,65.373130,0,761.189505,0,-5.502478,-37.725
166509,V20962517G,M-1009693J,o1V,2020-03-28,1.44,68,816,69,140.456738,0,4696.991810,0,25.604403,0.090


- `user_id` / `car_id` / `ride_id` — идентификаторы водителя, машины, поездки соответственно
- `ride_date` / `rating` — дата поездки и рейтинг, поставленный водителем
- `ride_duration` / `distance` / `ride_cost` —  длительность (время),  пройденное расстояние, стоимость поездки
- `speed_avg` / `speed_max` — средняя и максимальная скорости поездки соответственно
- `stop_times` / `refueling` — количество остановок (паузы) и флаг: была ли дозаправка
- `user_ride_quality` — оценка манеры вождения водителя машины, определенная скоринговой ML-системой сервиса
- `deviation_normal` — общий показатель датчиков о состоянии машины, относительно эталонных показателей (нормы)

### Информация про водителей

In [ ]:
driver_info = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/driver_info.csv')
driver_info.sample(5)

,age,user_rating,user_rides,user_time_accident,user_id,sex,first_ride_date
6645,34,6.9,942,1.0,V18323542E,0,2019-3-11
14052,37,7.9,1471,17.0,x18393623x,0,2021-6-3
13751,30,7.8,1150,11.0,I37981597Q,1,2020-12-12
7024,39,8.5,1408,5.0,D32465922h,0,2021-5-15
7590,40,9.0,369,13.0,Q41656989D,0,2019-8-10


- `user_id` / `age` / `sex` — идентификатор, возраст и пол водителя, соответственно
- `user_rating` — общий рейтинг пользователя за все поездки к концу 2021 года
- `user_rides` — общее количество поездок к концу 2021 года
- `user_time_accident` — число инцидентов (это могли быть аварии/штрафы/эвакуация машины)  
- `first_ride_date` — дата первой поездки

### Информация про ремонт машин

In [ ]:
fix_info = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/fix_info.csv')
fix_info.sample(5)

,car_id,worker_id,fix_date,work_type,destroy_degree,work_duration
22442,Z11671085y,CT,2019-5-16 21:58,reparking,1.0,26
8624,T65731704H,SM,2020-2-8 7:43,repair,7.4,38
130218,C-1531638p,XK,2019-6-28 5:25,reparking,1.0,27
57026,d-6273123g,OF,2020-1-29 8:14,refuel_check,1.0,28
38838,e47878499e,AS,2018-1-29 22:10,reparking,1.0,26


- `worker_id` / `car_id` — идентификатор работника и машины
- `work_type` / `work_duration` — тип и длительность (в часах) проводимой работы
- `destroy_degree` — степень износа/поврежденности машины в случае поломки
- `fix_date` — время начала ремонта (снятия машины с линии)

# Домашнее задание 1. Генерация и фильтрация признаков

<center> <img src="https://ucarecdn.com/bf4c772d-b67a-42ae-a48b-cfd83910b0a2/" width=700>

<div class="alert alert-info">

**Цель блока** — сгенерировать признаки из дополнительных датасетов и добавить их к основному датасету, произвести фильтрацию признаков.

    
## Задание 1. Генерация признаков из дополнительных датасетов (6 баллов)

<div class="alert alert-info">

1. Возьмите датасет `rides_info` с информацией о поездках и проведите группировку по каждой машине отдельно.

2. Для каждой машины (то есть для каждого `car_id`) подсчитайте несколько признаков:
  - минимальный рейтинг за все поездки;
  - средний рейтинг за все поездки;
  - общее количество километров, которое машина проехала;
  - максимальная скорость;
  - общее количество поездок для каждой машины.

3. Добавьте минимум 3 признака на свой выбор.

4. Сделайте соединение таблиц — вспомните про методы соединения и выберите подходящий для этого случая.

5. Подключите информацию про водителей (`driver_info`) и про ремонт машин (`fix_info`). Добавьте минимум 3 любых признака с каждого датасета.


<h4> Критерии оценивания </h4>
    
- Добавлены 5 обязательных и минимум 3 любых признака из датасета `rides_info` — 2 балла.
- Добавлены минимум 3 любых признака из датасета `driver_info` — 2 балла.
- Добавлены минимум 3 любых признака из датасета `fix_info` — 2 балла.

</div>

<div class="alert alert-success">
    
<h4> Советы по Feature Engineering</h4>
    
- Начинайте с сырых данных.
- Используйте все доступные данные. Покрывайте признаками всю имеющуюся информацию в данных.
- Формулируйте предположения: от чего зависит таргет?
- Смотрите визуально на классы/ошибки и делайте предположения. Какие полезны?
- Помните, что слишком много признаков может быть вредно. Потом придется отфильтровывать.
    
<h4> Полезные источники </h4>
    
- Занятие про Feature Engineering.
- Занятие про Feature Selection.
- [Max Kuhn and Kjell Johnson. Feature Engineering and Selection: A Practical Approach for Predictive Models](http://www.feat.engineering/).

### Пример расчета признака

In [ ]:
# Пример расчета одного признака
# rides_info.groupby('car_id', as_index=False).agg(
#     mean_rating = ('rating', 'mean'),
#     # ... еще признаки
# )

# df = pd.merge(...) # Соедините полученный датасет с фичами с основным датасетом

# YOUR CODE HERE
rides_grouped = rides_info.groupby('car_id').agg(
    min_rating=('rating', 'min'),
    mean_rating=('rating', 'mean'),
    total_distance=('distance', 'sum'),
    max_speed=('speed_max', 'max'),
    total_rides=('ride_id', 'count'),
    user_id=('user_id', lambda x: list(x))
)


rides_grouped['mean_duration'] = rides_info.groupby('car_id')['ride_duration'].mean()
rides_grouped['mean_cost'] = rides_info.groupby('car_id')['ride_cost'].mean()
rides_grouped['mean_user_ride_quality'] = rides_info.groupby('car_id')['user_ride_quality'].mean()

rides_grouped.reset_index(inplace=True)

print(rides_grouped.head())


       car_id  min_rating  mean_rating  total_distance   max_speed  \
0  A-1049127W         0.1     4.255172    1.125753e+07  179.729652   
1  A-1079539w         0.1     4.088046    1.912765e+07  184.505566   
2  A-1162143G         0.1     4.662299    2.995194e+06  180.000000   
3  A-1228282M         0.1     4.225172    1.793685e+07  182.446070   
4  A-1339912r         0.1     4.690115    1.202552e+07  152.000000   

   total_rides                                            user_id  \
0          174  [o52317055h, H41298704y, v88009926E, t14229455...   
1          174  [k91911033h, h75950158f, K14711042t, P20456464...   
2          174  [X21167121X, o11160326l, w48163041v, y64330631...   
3          174  [F17845906t, u23043461b, B15265668y, c88986152...   
4          174  [X12967899W, s81460917M, q19715188u, t12548491...   

   mean_duration     mean_cost  mean_user_ride_quality  
0    1289.034483  15201.436782                0.226325  
1    2148.810345  27007.477011                1.42

In [ ]:
rides_grouped_exploded = rides_grouped.explode('user_id')
merged_data = pd.merge(rides_grouped_exploded, driver_info, on='user_id', how='left')
merged_data = pd.merge(merged_data, fix_info, on='car_id', how='left')
print(merged_data.head())


       car_id  min_rating  mean_rating  total_distance   max_speed  \
0  A-1049127W         0.1     4.255172    1.125753e+07  179.729652   
1  A-1049127W         0.1     4.255172    1.125753e+07  179.729652   
2  A-1049127W         0.1     4.255172    1.125753e+07  179.729652   
3  A-1049127W         0.1     4.255172    1.125753e+07  179.729652   
4  A-1049127W         0.1     4.255172    1.125753e+07  179.729652   

   total_rides     user_id  mean_duration     mean_cost  \
0          174  o52317055h    1289.034483  15201.436782   
1          174  o52317055h    1289.034483  15201.436782   
2          174  o52317055h    1289.034483  15201.436782   
3          174  o52317055h    1289.034483  15201.436782   
4          174  o52317055h    1289.034483  15201.436782   

   mean_user_ride_quality  ...  user_rating  user_rides  user_time_accident  \
0                0.226325  ...          8.0         228                 9.0   
1                0.226325  ...          8.0         228           

### Идеи для новых признаков

1.   Из датасета rides_info:

    - Среднее время поездки за все поездки машины.
    - Среднее расстояние за одну поездку.
    - Количество поездок, в которых были остановки (паузы).
    - Доля поездок с дозаправкой (поле refueling).
    - Средний рейтинг, который водители ставили за поездки на данной машине.
    - Общее количество пользователей, которые использовали данную машину.

2.  Из датасета driver_info:

    - Среднее время с момента первой поездки на данной машине.
    - Количество инцидентов на каждую машину (user_time_accident).
    - Средний рейтинг, который водитель получил за все поездки на данной машине.

3.  Из датасета fix_info:

    - Общее количество ремонтов для каждой машины.
    - Среднее время между ремонтами.
    - Среднее время, требуемое на ремонт машины.



In [ ]:
# YOUR CODE HERE
# Из предложенных признаков, наиболее полезными могут быть те, которые содержат информацию о различных аспектах
# состояния машины или поведении водителя, которая может быть связана с вероятностью возникновения поломок или других проблем
merged_data['driver_age'] = merged_data['age']
merged_data['total_user_rating'] = merged_data['user_rating']
merged_data['driver_total_rides'] = merged_data['user_rides']

merged_data['fix_degree'] = merged_data['destroy_degree']
merged_data['fix_type'] = merged_data['work_type']
merged_data['fix_duration'] = merged_data['work_duration']

print(merged_data.head())

       car_id  min_rating  mean_rating  total_distance   max_speed  \
0  A-1049127W         0.1     4.255172    1.125753e+07  179.729652   
1  A-1049127W         0.1     4.255172    1.125753e+07  179.729652   
2  A-1049127W         0.1     4.255172    1.125753e+07  179.729652   
3  A-1049127W         0.1     4.255172    1.125753e+07  179.729652   
4  A-1049127W         0.1     4.255172    1.125753e+07  179.729652   

   total_rides     user_id  mean_duration     mean_cost  \
0          174  o52317055h    1289.034483  15201.436782   
1          174  o52317055h    1289.034483  15201.436782   
2          174  o52317055h    1289.034483  15201.436782   
3          174  o52317055h    1289.034483  15201.436782   
4          174  o52317055h    1289.034483  15201.436782   

   mean_user_ride_quality  ...          fix_date     work_type  \
0                0.226325  ...   2020-10-14 14:0     reparking   
1                0.226325  ...   2019-6-27 16:56  refuel_check   
2                0.226325 

* `feature_min_max_diff` — разница между максимальным и минимальным значениями `deviation_normal` для каждой машины
* `feature_corner` — угол наклона по признаку `user_ride_quality` для каждой машины
* `feature_mean` — среднее значение `deviation_normal` для каждой машины
* `feature_shift` — точка перегиба/сдвига для `deviation_normal`
* `feature_start` — значение точки старта для `deviation_normal`
* `feature_nans` — сумма пропусков для столбца `...` для каждой машины
* `feature_quant` — `X %` квантиль для столбца `...` для каждой машины

Подумайте, какие из признаков могут оказаться самыми полезными?

## Задание 2. Применение методов фильтрации признаков (4 балла)

<center> <img src="https://ucarecdn.com/d1b4bc78-fd04-44fb-bdbf-0a63355b7384/" width=700>

1. Выберите и примените любые 3 (можно и больше) метода фильтрации, которые указаны выше или в ноутбуке по фильтрации из полезных ссылок.
2. Проинтерпретируйте результаты и сделайте выводы.


<h4> Критерии оценивания </h4>
    
- Применены методы фильтрации — 1 балл за каждый метод, максимум 3 балла.
- Сделаны выводы на основе примененных методов —1 балл.
</div>

### Советы по Feature Selection

<div class="alert alert-info">

<h3><center>Зачем отбирать признаки</center></h3>
    
**Основные причины**:
    
- Главная причина: с увеличением количества признаков часто падает точность предсказания модели, а некоторые модели и вовсе перестают адекватно работать. Так происходит, если в данных большое количество мусорных фичей (почти не коррелирующих с таргетом).

- Если фичей очень много, то данные перестают помещаться в память и существенно увеличивают время обучения модели, особенно если мы тестируем несколько алгоритмов или ансамбль. Также важно учитывать, что платформы имеют ограничения на длительность одной сессии (в Kaggle — 12 часов) и лимиты по потребляемой памяти.

- Даже если точность не снижается, есть риск, что модель опирается на шумные фичи, что снизит стабильность прогноза на приватной выборке.


<b>Что удалить сразу:</b>
    
- константы;
- уникальные значения (в том числе в тесте; как правило, это ID-шники по типу `car_id`).
    
<b>Какие методы использовать:</b>
    
- `Линейная корреляция`.
- `Phik` — тоже корреляция, но на стероидах ([туториал на Medium](https://towardsdatascience.com/phik-k-get-familiar-with-the-latest-correlation-coefficient-9ba0032b37e7)).
- `Permutation importance` (из `scikit-learn`).
- `SHAP values` (из библиотеки `shap`).
- `CatBoost Feature Selection` (рекурсивные методы).
    

In [ ]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 3.4 MB/s eta 0:00:00


In [ ]:
# YOUR CODE HERE
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
import shap
used_features = ["min_rating",  "mean_rating",  "total_distance",   "max_speed"]
unused_features = set(merged_data.columns) - set(used_features)
merged_data.drop(unused_features, axis=1, inplace=True)

# Линейная корреляция
correlation_matrix = merged_data.corr()
print("Линейная корреляция с целевой переменной:\n", correlation_matrix)



Линейная корреляция с целевой переменной:
                 min_rating  mean_rating  total_distance  max_speed
min_rating        1.000000     0.382246       -0.051616  -0.287692
mean_rating       0.382246     1.000000       -0.136768  -0.592398
total_distance   -0.051616    -0.136768        1.000000   0.129074
max_speed        -0.287692    -0.592398        0.129074   1.000000


In [ ]:
# # Permutation importance
X = merged_data.drop(['max_speed'], axis=1)
y = merged_data['max_speed']
model = RandomForestRegressor(random_state=42)
model.fit(X, y)
perm_importance = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nPermutation importance:\n", perm_importance)


Permutation importance:
 mean_rating       0.715356
total_distance    0.209983
min_rating        0.074661
dtype: float64


In [ ]:
# # SHAP values
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
shap_importance = np.abs(shap_values).mean(axis=0)
shap_importance = pd.Series(shap_importance, index=X.columns).sort_values(ascending=False)
print("\nSHAP values:\n", shap_importance)

Выводы:

1)   Линейная корреляция с целевой переменной:

  - min_rating имеет положительную корреляцию с mean_rating, что означает, что более низкие оценки чаще встречаются вместе с более высокими оценками.

  - mean_rating имеет относительно сильную положительную корреляцию с min_rating и отрицательную корреляцию с max_speed. Это может указывать на то, что более высокие средние оценки чаще встречаются с более высокими минимальными оценками и более низкими максимальными скоростями.


  - total_distance имеет слабую положительную корреляцию с max_speed, что может означать, что более длинные расстояния чаще встречаются с более высокими максимальными скоростями.

2)    Permutation importance:
- mean_rating имеет самый высокий показатель важности. Это означает, что изменение этого признака приводит к наибольшим изменениям в предсказаниях модели. Таким образом, средняя оценка является наиболее важным признаком для модели.

- total_distance имеет второй по важности показатель, но он значительно ниже, чем у mean_rating.

- min_rating имеет наименьший показатель важности, что означает, что изменения в этом признаке вносят наименьший вклад в предсказания модели.

В целом, можно сделать вывод, что средняя оценка (mean_rating) является наиболее важным признаком для предсказания целевой переменной, а также что она имеет наибольшую корреляцию с другими признаками. Расстояние (total_distance) также оказывает некоторое влияние, хотя и меньше, чем средняя оценка. Минимальная оценка (min_rating) имеет меньшее значение и важность по сравнению с другими признаками.

# Домашнее задание 2. Прогноз времени и вида поломки машин. Настройка ML-модели

<div class="alert alert-info">

**Цель блока** — составить тренировочную и валидационную выборки, произвести обучение модели `CatBoostClassifier` на тренировочной части и оценить качество на валидационной.

##Обучение первой модели (5 баллов)

1.   Новый пункт
2.   Новый пункт



1. Классифицируйте признаки на типы (категориальные, числовые, таргеты).

2. Выделите в `X` только признаки, а в `y` таргет (для задачи классификации).

3. Сделайте разделение данных на *обучающую* и *валидационную* выборки (не забывайте про воспроизводимость результатов).

4. Создайте и обучите `CatBoostClassifier` модель (настраивать гиперпараметры сейчас не обязательно).

5. Проведите оценку вашей модели, используя метрику `accuracy`.


<h4> Критерии оценивания </h4>
    
- По 1 баллу за каждый корректно выполненный пункт.

</div>

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
features2drop = [...] # то, что надо выбросить
targets = [...] # таргеты
cat_features = [...] # категориальные признаки

num_features = [...] # числовые признаки

print('Категориальные признаки:', len(cat_features), cat_features)
print('Числовые признаки:', len(num_features), num_features)
print('Целевые переменные', targets)

In [ ]:
# YOUR CODE HERE

# Дополнительное задание. Визуализация прогнозов, ошибок модели и важности признаков

<div class="alert alert-info">

**Цель блока** — визуализировать результаты прогнозирования, ошибки модели и важность признаков для лучшего понимания и анализа модели.

## Визуализация (5 баллов)

   
1. Визуализируйте важность признаков встроенным методом в `CatBoost`  (помните, что не стоит сильно доверять этому рейтингу важности, т. к. для сильно скоррелированных признаков важность делится пополам — оба признака могут улететь вниз).

2. Постройте `waterfall_plot` из библиотеки `SHAP` (подробнее во втором ноутбуке из полезных ссылок ниже).
    
3. Постройте `classification_report` из библиотеки `scikit-learn`.

4. Постройте и визуализируйте матрицу смежности (`confusion_matrix`), посмотрите, в каких классах модель больше всего ошибается.
    
5. Для каждого графика/примененного метода проинтерпретируйте результаты и сделайте выводы.


<h4> Критерии оценивания </h4>
    
- По 1 баллу за каждый корректно выполненный пункт.
    
</div>

<div class="alert alert-success">

Визуализация может помочь даже после того, как мы **уже обучили** какую-нибудь модель. Например:

- Понять, что мешает модели или чего не хватает, чтобы не допускать ошибки.
- Сделать выводы, как можно улучшить точность в последующих экспериментах.
- Визуализировать ошибки модели.
- Отсеять лишние признаки.
- Найти идеи для новых признаков.
- Все зависит от типа ваших данных.

<h3> Полезные источники </h3>
    
- Занятие про визуализацию и Seaborn.
- Занятие про продвинутую визуализацию и анализ ошибок модели.

</div>    

In [ ]:
# YOUR CODE HERE

# Домашнее задание 3. Оптимизация. Тюнинг гиперпараметров с помощью `Optuna`

<div class="alert alert-info">

**Цель блока** — улучшить качество предсказания, произведя подбор гиперпараметров для модели с помощью `Optuna`.

## Подбор гиперпараметров (3 балла)

1. Напишите `objective` функцию и запустите `Optuna`.
2. Подберите гиперпараметры для `CatBoostClassifier` (минимум 3).
3. Обучите модель с новыми гиперпараметрами, сравните качество и сделайте выводы.

    
<h4> Критерии оценивания </h4>
    
- 1 балл за каждый корректно выполненный пункт.
    
</div>

In [ ]:
# YOUR CODE HERE

### Общая информация по `Optuna`

#### Ключевые особенности <code>Optuna</code>



- Легковесность и универсальность — можно подбирать оптимальные параметры под любые функции и метрики.
- SOTA-алгоритмы, адаптированные для поиска гиперпараметров.
- Параллелизация и различные методы прунинга.
- Встроенная визуализация.
- Интеграция со множеством популярных библиотек (бустинги, sklearn, PyTorch, W&B и другие).

Разберем Optuna по частям, чтобы понять, как ее использовать.

#### <code>Study</code>: оптимизация, базирующаяся на <code>Objective</code> функции



В `Objective`-функцию нужно написать код подсчета метрики, которую возвращаем. `Objective` вызывается Optuna много раз для подбора лучших параметров.
```python
def objective(trial, ...):
    # calculate score...
    return score
```

<h4> <code>Trial</code> — одно выполнение <code>Objective</code>-функции</h4>

В `trial`-объекте мы передаем параметры для «перебора», используя для каждого типа свой метод. Например:

```python
# метод `suggest_float` показывает, что перебираем `float` значения, от 0 и до 1.5 границы
param = trial.suggest_float('param', 0, 1.5)

# Категориальное значение
loss_function = trial.suggest_categorical('loss', ['Logloss', 'CrossEntropy'])

# Целочисленное значение
depth = trial.suggest_int('depth', 5, 8)

# Равномерное распределение
learning_rate = trial.suggest_uniform('learning_rate', 0.0, 1.0)
```

#### `Study` parameters

Инициализируем объект `study`, который начнет перебор и сохранит в себе историю результатов.
Если мы стараемся увеличить метрику, а не уменьшить ошибку, то используем `create_study(direction='maximize')`.
```python
study = optuna.create_study()
study.optimize(objective, n_trials=10)
```

<div class="alert alert-info">
    
В [`Optuna`](https://optuna.readthedocs.io/en/stable/index.html) реализовано несколько методов (`sampler`) подбора параметров (в том числе классические):
* `GridSampler`
* `RandomSampler`
* `Tree-Structed Parzen Estimator` (`TPESampler` — самый популярный, дефолтный)
* `BruteForceSampler`
* [Другие 4 метода](https://optuna.readthedocs.io/en/stable/reference/samplers/index.html#module-optuna.samplers) (можно написать собственный семплер)

### Советы по перебору параметров

- Иметь понимание важности параметров.
- Число `iterations` лучше взять с запасом и зафиксировать, при этом ограничив через `early_stopping_rounds`.
- Подсмотреть или чувствовать диапазоны и шаг значений.
- Исключить то, что перебирать не нужно (`random_seed` , `eval_metric`, `thread_count` и прочее).
- Используйте информацию с прошлых попыток.

# Домашнее задание 4. Блендинг

<div class="alert alert-info">

**Цель блока** — улучшить качество предсказания, объединив несколько моделей вместе методом блендинга.

## Задание 6. Блендинг (10 баллов)

1. Построить и обучить модели:
    - `CatBoostClassifier`,
    - `LightGBMClassifier (goss)`,
    - `XGBoostClassifier (dart)`,
    - `RandomForestClassifier`.
2. Сделать предсказания каждой моделью, оценить качество.
3. Реализовать блендинг двумя способами, оценить качество и сравнить с предыдущим пунктом:
    - `Hard Voting` —  метод, при котором делаем голосование всех моделей и выбираем самый популярный класс.
    - `Soft Voting` —  метод, при котором мы складываем вероятности предсказания всех моделей по каждому классу и потом выбираем самый класс с максимальной суммой.
    
<h4> Критерии оценивания </h4>
    
- Обучены 4 модели — 1 балл за каждую обученную модель, всего 4 балла.
- Для модели проведен подбор гиперпараметров — 0,5 балла за каждый параметр, всего 2 балла.
- Сделаны предсказания каждой модели и оценено качество — 1 балл.
- Реализован `Hard Voting` — 1 балл.
- Реализован `Soft Voting` — 1 балл.
- Сделаны выводы — 1 балл.
    
</div>

In [ ]:
# YOUR CODE HERE

### Общая информация о блендинге

**Основная идея** — взять от каждого алгоритма лучшее и совместить несколько разных ML-моделей в одну.

Что это дает:
- Увеличивается обобщающая способность финальной модели и качество улучшается.
- Модель становится более стабильной, что позволяет не слететь на приватном лидерборде.

Особенно хорошо накидывает блендинг, если смешиваемые модели имеют разную природу — например, нейронные сети, kNN и решающие деревья. Они выучивают разные зависимости и хорошо дополняют друг друга.

</div>

# <center id="part6"> Отправка финального решения на лидерборд на Kaggle

<div class="alert alert-info">

**Цель блока** — сделать предсказание с помощью блендинга для тестовой части датасета, отправить результат [в соревнование на Kaggle](https://www.kaggle.com/competitions/competative-data-science-course-by-data-feeling/overview).

## Задание 8. Предсказание на тестовом датасете и отправка на Kaggle

1. Сделать предобработку для тестового датасета, присоединить к нему информацию из других датасетов и добавить признаки, которые генерировали для тренировочного датасета.
2. Сделать предсказания каждой моделью, которую хотите включить в ансамбль, но минимум их должно быть 3.
3. Сделать блендинг с помощью `Hard Voting` или `Soft Voting` на ваш выбор.
4. Сохранить результат предсказания в `csv` файл и отправить решение на Kaggle.
    
<h4> Критерии оценивания </h4>
    
- 0 баллов за задание, если итоговый скор на лидерборде меньше чем `0.9`.
- 1 балл за каждый выполненный пункт.

</div>

In [ ]:
car_test = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/car_test.csv')
car_test.head(3)

In [ ]:
# YOUR CODE HERE

### Советы по отправке кода на Kaggle



Если пишете код на `Kaggle`, то для отправки решения:

1. Нажать Save Version.
2. Проверить, что в Advanced Settings выбрано «Всегда сохранять результаты ноутбука».
3. Выбрать тип сохранения Save & Run All (Commit).
4. Нажать кнопку Save.